In [1]:
# Convert .json to .csv

# https://gist.github.com/paulgb/5265767
import json
import pandas as pd
from glob import glob

def convert(x):
    ''' Convert a json string to a flat python dictionary
    which can be passed into Pandas. '''
    ob = json.loads(x)
    for k, v in ob.items():
        if isinstance(v, list):
            ob[k] = ','.join(v)
        elif isinstance(v, dict):
            for kk, vv in v.items():
                ob['%s_%s' % (k, kk)] = vv
            del ob[k]
    return ob

for json_filename in glob('*.json'):
    csv_filename = '%s.csv' % json_filename[:-5]
    print 'Converting %s to %s' % (json_filename, csv_filename)
    df = pd.DataFrame([convert(line) for line in file(json_filename)])
    df.to_csv(csv_filename, encoding='utf-8', index=False)

Converting yelp_academic_dataset_business.json to yelp_academic_dataset_business.csv
Converting yelp_academic_dataset_checkin.json to yelp_academic_dataset_checkin.csv
Converting yelp_academic_dataset_review.json to yelp_academic_dataset_review.csv
Converting yelp_academic_dataset_tip.json to yelp_academic_dataset_tip.csv
Converting yelp_academic_dataset_user.json to yelp_academic_dataset_user.csv


TypeError: sequence item 0: expected string, int found

In [4]:
tips = pd.read_csv('yelp_academic_dataset_tip.csv')
tips = tips.drop('type',axis=1)
tips.head()

,business_id,date,likes,text,user_id
0,5UmKMjUEUNdYWqANhGckJw,2016-07-17,0,Pizza is garbage. Hoagies are excellent.,ZxdojCZgZhw36xuSGaryRg
1,cE27W9VPgO88Qxe4ol6y_g,2013-04-18,0,Don't waste your time.,-6rEfobYjMxpUWLNxszaxQ
2,mVHrayjG3uZ_RLHkLj-AMg,2016-02-22,0,Not easy to find. Be sure to put in directions...,5OumCBQ_MyQsltSdbMyunA
3,mVHrayjG3uZ_RLHkLj-AMg,2013-01-06,1,Your GPS will not allow you to find this place...,EZ0r9dKKtEGVx2CdnowPCw
4,KayYbHCt-RkbGcPdGOThNg,2013-12-03,0,Great drink specials!,xb6zEQCw9I-Gl0g06e1KsQ


In [7]:
#reviews = pd.read_csv('yelp_academic_dataset_review.csv')
reviews = reviews.drop('date',axis=1)
reviews = reviews.drop('review_id',axis=1)
reviews = reviews.drop('text',axis=1)
reviews = reviews.drop('type',axis=1)
reviews = reviews.drop('user_id',axis=1)
reviews = reviews.drop('votes_cool',axis=1)
reviews = reviews.drop('votes_funny',axis=1)
reviews = reviews.drop('votes_useful',axis=1)
reviews.head()

,business_id,stars
0,5UmKMjUEUNdYWqANhGckJw,4
1,5UmKMjUEUNdYWqANhGckJw,5
2,5UmKMjUEUNdYWqANhGckJw,5
3,5UmKMjUEUNdYWqANhGckJw,3
4,5UmKMjUEUNdYWqANhGckJw,2


In [16]:
reviews = reviews.groupby(['business_id'],as_index=False).mean()

In [20]:
reviews['stars'] = reviews['stars'].round()
reviews.head()

,business_id,stars
0,--0ZoBTQWQra1FxD4rBWmg,2
1,--1emggGHgoG6ipd_RMb-g,3
2,--4Pe8BZ6gj57VFL5mUE8g,3
3,--5jkZ3-nUPZxUvtcbr8Uw,5
4,--7PRjnsjMA6uhPK8mW13Q,3


In [23]:
tips = pd.merge(tips, reviews, on='business_id')
tips.head()

,business_id,date,likes,text,user_id,stars
0,5UmKMjUEUNdYWqANhGckJw,2016-07-17,0,Pizza is garbage. Hoagies are excellent.,ZxdojCZgZhw36xuSGaryRg,3
1,cE27W9VPgO88Qxe4ol6y_g,2013-04-18,0,Don't waste your time.,-6rEfobYjMxpUWLNxszaxQ,3
2,mVHrayjG3uZ_RLHkLj-AMg,2016-02-22,0,Not easy to find. Be sure to put in directions...,5OumCBQ_MyQsltSdbMyunA,5
3,mVHrayjG3uZ_RLHkLj-AMg,2013-01-06,1,Your GPS will not allow you to find this place...,EZ0r9dKKtEGVx2CdnowPCw,5
4,KayYbHCt-RkbGcPdGOThNg,2013-12-03,0,Great drink specials!,xb6zEQCw9I-Gl0g06e1KsQ,4


In [24]:
tips.to_csv('tips.csv',index=False)